In [57]:
import pandas as pd
import numpy as np

In [58]:
df = pd.read_csv('../data/Modelar_UH2021.txt', sep="|")

In [59]:
df.head()

,fecha,id,visitas,categoria_uno,categoria_dos,estado,precio,dia_atipico,campaña,antiguedad,unidades_vendidas
0,1/6/2015 0:00:00,21972,0,C,75.0,No Rotura,NaN,0,0,5241.0,0
1,1/6/2015 0:00:00,23910,5,C,170.0,No Rotura,"6,07",0,0,5241.0,3
2,1/6/2015 0:00:00,24306,13,A,46.0,No Rotura,NaN,0,0,NaN,0
3,1/6/2015 0:00:00,24306,13,A,46.0,No Rotura,NaN,0,0,NaN,0
4,1/6/2015 0:00:00,27144,15,E,230.0,No Rotura,NaN,0,0,4064.0,0


In [60]:
df.shape

(4045022, 11)

In [61]:
df.dtypes

fecha                 object
id                     int64
visitas                int64
categoria_uno         object
categoria_dos        float64
estado                object
precio                object
dia_atipico            int64
campaña                int64
antiguedad           float64
unidades_vendidas      int64
dtype: object

Buscamos valores null en las columnas que forman el conjunto de datos.

In [62]:
missing_columns = df.columns[df.isna().any()].tolist()
missing_columns

['categoria_dos', 'precio', 'antiguedad']

Podemos observar que hay tres columnas con valores nulos. Ahora lo que vamos a hacer es ver con cuántos valores nulos cuenta cada una de ellas y de esta manera, decidir de qué forma tratarlos.

In [63]:
missing_data = df.isnull() # Los valores null se representan como True

for column in missing_columns:
    print(column)
    print (missing_data[column].value_counts())
    print("")

categoria_dos
False    4039178
True        5844
Name: categoria_dos, dtype: int64

precio
True     2642911
False    1402111
Name: precio, dtype: int64

antiguedad
False    3170857
True      874165
Name: antiguedad, dtype: int64



Cantidad de valores null por columna:
* `categoria_dos`: 5844
* `precio`: 2642911
* `antiguedad`: 874165

Para tratar los valores nulos de la columna `categoria_dos` vamos a reemplazarlos por el valor más frecuente de esta:

In [64]:
most_freq = df['categoria_dos'].mode().values[0]
most_freq

307.0

In [65]:
df['categoria_dos'].replace(np.nan, most_freq, inplace=True)

Para tratar los valores nulos de la columna `antiguedad` vamos a reemplazarlos por la media de esta columna:

In [66]:
mean_val = int(df['antiguedad'].mean())
mean_val

1011

In [67]:
df['antiguedad'].replace(np.nan, mean_val, inplace=True)

Para tratar los valores nulos de la columna `precio` vamos a realizar lo que se indica en las instrucciones del reto "*Cuando su valor es nulo, ha de ser completado con el precio anterior temporalmente más cercano para cada artículo.*"

In [68]:
# def nearest(items, pivot):
#     return min([i for i in items if i < pivot], key=lambda x: abs(x - pivot))

In [71]:
# Hay que encontrar la fila con NaN en precio. Coger su id y buscar las filas con el mismo id. Buscar la fecha anterior mas cercana y poner su precio

# DataFrame con las filas que no tienen precio
precio_nan = df[['fecha','id','precio']].where(df['precio'].isnull()).dropna(how='all')
precio_nan['fecha'] = pd.to_datetime(precio_nan['fecha'])
print(precio_nan)

# DataFrame con las filas que tienen precio
precio_not_nan = df[['fecha','id','precio']].dropna()
precio_not_nan['fecha'] = pd.to_datetime(precio_not_nan['fecha'])
print(precio_not_nan)

for index, row in precio_nan.iterrows():
    # Filas con precio que tienen el mismo id que la que no tiene precio
    aux1 = precio_not_nan.where(precio_not_nan['id'] == row['id']).dropna(how='all')
    print('Fecha escogida: ')
    print(row['fecha'])

    # Se anade una columna con la diferencia entre las fechas de la fila
    # que estamos mirando sin precio y las que tienen precio
    # Mirar si puede dar negativo
    # aux1['diferencia'] = (row['fecha'] - aux1['fecha']).dt.days
    
    fecha_anterior = aux1['fecha'].where(aux1['fecha'] < row['fecha']).dropna(how='all')
    print('--------------------------')
    print(fecha_anterior)
    
    print('Fecha anterior mas reciente: ')    
    print(aux1['fecha'].truncate(before=str(row['fecha'])).head())
    
    
    # Seleccionamos la menor 


             fecha        id precio
0       2015-01-06   21972.0    NaN
2       2015-01-06   24306.0    NaN
3       2015-01-06   24306.0    NaN
4       2015-01-06   27144.0    NaN
5       2015-01-06   27504.0    NaN
...            ...       ...    ...
4044978 2016-09-30  437904.0    NaN
4044997 2016-09-30  447074.0    NaN
4045011 2016-09-30  452730.0    NaN
4045018 2016-09-30  457422.0    NaN
4045019 2016-09-30  458650.0    NaN

[2642911 rows x 3 columns]
             fecha      id precio
1       2015-01-06   23910   6,07
6       2015-01-06   30014   6,12
8       2015-01-06   31180   8,05
10      2015-01-06   35732  26,24
14      2015-01-06   40850  14,18
...            ...     ...    ...
4045015 2016-09-30  454950  19,23
4045016 2016-09-30  456982  81,28
4045017 2016-09-30  457416  50,38
4045020 2016-09-30  458660  68,49
4045021 2016-09-30  458660  68,49

[1402111 rows x 3 columns]


KeyboardInterrupt: 

In [216]:
precio_not_nan = df[['fecha','id','precio']].dropna()
precio_not_nan

,fecha,id,precio
1,1/6/2015 0:00:00,23910,"6,07"
6,1/6/2015 0:00:00,30014,"6,12"
8,1/6/2015 0:00:00,31180,"8,05"
10,1/6/2015 0:00:00,35732,"26,24"
14,1/6/2015 0:00:00,40850,"14,18"
...,...,...,...
4045015,30/9/2016 0:00:00,454950,"19,23"
4045016,30/9/2016 0:00:00,456982,"81,28"
4045017,30/9/2016 0:00:00,457416,"50,38"
4045020,30/9/2016 0:00:00,458660,"68,49"


In [217]:
df.shape

(4045022, 11)